# 01_LangChain Setup with Local Ollama Container Instance

This notebook demonstrates how to set up LangChain to work with a local Ollama container instance.
We will cover the installation of necessary packages, configuration, and connection to the local Ollama instance.
This could be useful for using LangChain/LangFlow like techniques.

## Step 1: Install Necessary Packages

First, we need to install the required packages. Run the following command to install LangChain dependencies.

Typically, this is already done with a `pip install -r requirements.txt`, and is only needed once.

In [ ]:
%pip install -q langchain langchain_community langchain_ollama

## Step 2: Import Packages

Next, we will import the necessary packages for our setup.

In [ ]:
import requests
from typing import TypedDict
from langchain_ollama import OllamaLLM, ChatOllama

## Step 3: Configure Connection to Local Ollama Instance

We need to configure the connection to our local Ollama container instance. The following code sets up the connection.

### Step 3.1: Setup Ollama Docker Container Instance

*_Run the following step first. This step is only needed if the following step fails._*

#### Ollama
Ollama is a containerized environment for running and managing LLMs. It provides an API for interacting with the models.

#### Setup Instructions
1. Ensure Docker is installed and running on your machine.
2. Check if there is a Docker container instance 'ollama' that can be (re-)started.
3. _If no 'ollama' container exists_: Create and run a new Ollama container instance using the following command:
   ```
   docker run -d --gpus=all -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama
   ```
4. Verify the Ollama instance is running by accessing [http://localhost:11434/api/version](http://localhost:11434/api/version).
5. Verify programmatic connection by (re-)running the following cell:


In [ ]:
OLLAMA_API_URL = "http://localhost:11434/api"


def get_ollama_version():
    response = requests.get(f"{OLLAMA_API_URL}/version")
    if response.status_code == 200:
        return response.json()
    else:
        return None


try:
    ollama_version = get_ollama_version()
    if ollama_version:
        print(f"Connected to Ollama version: {ollama_version}")
    else:
        print("Failed to connect to Ollama instance.")
except requests.exceptions.ConnectionError:
    print("Failed to connect to Ollama instance, is the Docker container running?")
    input("Press Enter to continue...")
    raise

## Step 4: Minimal Documentation and Instructions

### LangChain
LangChain is a framework for building applications with large language models (LLMs). It provides tools and abstractions to simplify the development process.

https://python.langchain.com/docs/tutorials/


## Step 5: Demonstrate LangChain

In this step, we will demonstrate basic LangChain usage.

### Example: Text Generation

Use LangChain to generate text based on a prompt.



In [ ]:
model = OllamaLLM(model="llama3.2")
generated_text = model.invoke("Come up with 10 names for a song about parrots")

generated_text

### References

- [LangChain Documentation](https://python.langchain.com/docs/)

### Furture Work Suggestion

- [LangChain and DSpy Integration](https://www.reddit.com/r/LangChain/comments/1cqexk6/thoughts_on_dspy/)

## Step 6: User Input for Software Project Idea Using LangChain

In this step, we will prompt the user to write a software project idea, send it to the LLM, and display the feedback, summary, and plan.

### Example: User Input and Feedback Loop

1. Prompt the user to write a software project idea.
2. Send the idea to the LLM and display the feedback, summary, and plan.
3. Implement a refinement loop to allow the user to provide additional input and receive updated feedback.

#### Prompt User for Software Project Idea
We will prompt the user to write a software project idea.

In [ ]:
# 7.1 Get user input for software project idea, default to
project_idea = (
    input("Please write your software project idea: ")
    or "Create an AI software factory that generates software from project ideas, using step-by-step software processes that are implemented by LLM Agents."
)
print(project_idea)

In [ ]:
prompt_template = """A user came up with a project idea: {project_idea}

The user wants to know what the project would look like if it was implemented.

Write a short description of the project, including the main features and how it would work.

Then give your feedback on the project idea, including any suggestions for improvement, or
areas that may need clarification.
"""

In [ ]:
filled_prompt = prompt_template.format(project_idea=project_idea)
print(filled_prompt)

In [ ]:
project_idea_feedback = model.invoke(filled_prompt)
print(project_idea_feedback)

### Step 7. Implement Refinement Loop

We will implement a refinement loop to allow the user to provide additional input and receive updated feedback.

In [ ]:
while True:
    additional_input = input(
        f"""
                             Please provide additional input to refine your project idea (or type 'exit' to finish):
                             
                             {project_idea}
                             """
    )
    if not additional_input.strip():
        break

    refinement_prompt_template = """
    You are an AI assistant. The user has provided a project description and additional input to refine the project idea.
    Provide an updated project description that addresses the user's input.

    {project_idea}

    Additional Input: {additional_input}
    """

    refinement_prompt = refinement_prompt_template.format(project_idea=project_idea, additional_input=additional_input)
    updated_feedback_summary_plan = model.invoke(refinement_prompt)
    print(updated_feedback_summary_plan)

## Step 8: Add TypedDict Joke Class

We will add a `TypedDict` class `Joke` with fields `setup` and `punchline`.

In [ ]:
from typing import Optional

from pydantic import BaseModel, Field


# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke, e.g. a riddle question")
    punchline: str = Field(description="The punchline to the joke, e.g. a funny answer")
    rating: Optional[int] = Field(default=None, description="How funny the joke is, from 1 to 10")

In [ ]:
from langchain_ollama import ChatOllama

chat_ollama = ChatOllama(
    model="mistral",
    temperature=0.7,
    # other params...
)
structured_llm = chat_ollama.with_structured_output(Joke)

In [ ]:
joke = structured_llm.invoke("Tell me a joke about AI")
joke

## Step 9: Implement LangChain Chat Client

We will implement a LangChain chat client that connects to a local Ollama container instance.

In [ ]:
class LangChainChatClient:
    def __init__(self, model_name: str):
        self.model = OllamaLLM(model=model_name)
        self.structured_llm = self.model.with_structured_output(Joke)

    def generate_joke(self, prompt: str) -> Joke:
        response = self.structured_llm.invoke(prompt)
        print(response)
        joke = response.split("\n")
        return Joke(setup=joke[0], punchline=joke[1])

## Step 10: Demonstrate Chat Client Functionality

We will demonstrate the chat client's functionality by generating and displaying jokes using the local Ollama model.